# 멋진 작사가 만들기

## (1) 데이터 다운로드

In [1]:
# wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip 이용

## (2) 데이터 읽어오기

* glob 모듈 -  파일을 읽어오는 작업 용이

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Ghost in the mirror', "I knew your face once, but now it's unclear", "And I can't feel my body now"]


## (3) 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.



In [3]:
print(raw_corpus[:10])  

['Ghost in the mirror', "I knew your face once, but now it's unclear", "And I can't feel my body now", "I'm separate from here and now A drug and a dream", 'We lost connection, oh come back to me', 'So I can feel alive again', "Soul and body try to mend It's pulling me apart, this time", 'Everything is never ending', 'Slipped into a peril that', "I'll never understand"]


* 너무 긴 문장은 쉼표기준으로도 분리할 필요있어보임

In [4]:
import re

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제 - (대소문자 통일)

    # 특문 >> (공백)특문(공백) 안됨 -- r'\1'
    # 특문 >> 공백
    #()는 코러스인데 삭제하는게 낫지않을까
    sentence = re.sub(r"[(a-zA-Z,.)],"," ", sentence)
    sentence = re.sub(r"[.,]", r" ", sentence)        
        
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?'.!,¿]+", " ", sentence)  # 영어대소문자, 특문아닌 것(^)은 스페이스 1개로 치환
    #it's 같은거 살리기

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence <end>


In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 15: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> ghost in the mirror <end>',
 "<start> i knew your face onc but now it's unclear <end>",
 "<start> and i can't feel my body now <end>",
 "<start> i'm separate from here and now a drug and a dream <end>",
 '<start> we lost connectio oh come back to me <end>',
 '<start> so i can feel alive again <end>',
 "<start> soul and body try to mend it's pulling me apar this time <end>",
 '<start> everything is never ending <end>',
 '<start> slipped into a peril that <end>',
 "<start> i'll never understand <end>"]

In [7]:
corpus[:50]

['<start> ghost in the mirror <end>',
 "<start> i knew your face onc but now it's unclear <end>",
 "<start> and i can't feel my body now <end>",
 "<start> i'm separate from here and now a drug and a dream <end>",
 '<start> we lost connectio oh come back to me <end>',
 '<start> so i can feel alive again <end>',
 "<start> soul and body try to mend it's pulling me apar this time <end>",
 '<start> everything is never ending <end>',
 '<start> slipped into a peril that <end>',
 "<start> i'll never understand <end>",
 '<start> this feeling always gets away <end>',
 '<start> wishing i could hold on longer <end>',
 "<start> it doesn't have to feel so strange <end>",
 "<start> i knew your face onc but now it's unclear <end>",
 "<start> and i can't feel my body now <end>",
 "<start> i'm separate from here and now it's pulling me apar this time <end>",
 '<start> everything is never ending <end>',
 '<start> slipped into a peril that <end>',
 "<start> i'll never understand <end>",
 '<start> this fee

* 토크나이징

In [8]:
import tensorflow as tf

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

In [9]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  
        filters=' ',    
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 1373   12 ...    0    0    0]
 [   2    5  385 ...    0    0    0]
 [   2    7    5 ...    0    0    0]
 ...
 [   2   72   38 ...    3    0    0]
 [   2  103  754 ...    0    0    0]
 [   2   10  837 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f257aae87d0>


In [10]:
## 모델의 최종 출력 텐서 shape 어떻게 확인? (1760000개 문장이 최대 15줄, 5000개 단어사용했는데)

In [11]:
tensor.shape

(166092, 15)

In [12]:
print(tensor[:3, :10]) # 3행,10열까지만

[[   2 1373   12    4  864    3    0    0    0    0]
 [   2    5  385   17  218 3527   27   48   50 8041]
 [   2    7    5   79  101   10  243   48    3    0]]


* 단어사전 형태확인

In [13]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : a
9 : to
10 : my


In [14]:
# 2로 시작해서 3으로 끝나거나, 길어서 안끝나거나..

In [15]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.


print(src_input[0])
print(tgt_input[0])

[   2 1373   12    4  864    3    0    0    0    0    0    0    0    0]
[1373   12    4  864    3    0    0    0    0    0    0    0    0    0]


* 데이터셋 객체 생성  
 실제문장과 다르게 시작과 끝만 주고 나머지는 모델이 학습하면서 채워넣게 함

In [16]:
src_input.shape, tgt_input.shape

((166092, 14), (166092, 14))

In [17]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 1024
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # 사전 + 포함되지 않은 0:<pad>를 포함하여 5001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((1024, 14), (1024, 14)), types: (tf.int32, tf.int32)>

## (4) 평가 데이터셋 분리

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

In [20]:
# Source Train: (124960, 14)
# Target Train: (124960, 14)

In [21]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print(enc_val.shape)
print(dec_val.shape)
# 더 어떻게 줄이는지 모르겠음

Source Train: (132873, 14)
Target Train: (132873, 14)
(33219, 14)
(33219, 14)


## (5) 인공지능 만들기

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 512
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [23]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(1024, 14, 12001), dtype=float32, numpy=
array([[[ 3.77360877e-04,  1.32223759e-05,  4.77482463e-05, ...,
         -6.39169375e-05,  3.30271178e-05, -3.16581660e-04],
        [ 5.19942492e-04, -1.48055507e-04, -6.53131647e-05, ...,
         -2.70952096e-05,  5.41001245e-05, -4.46536491e-04],
        [ 6.15278492e-04, -1.29224733e-04, -1.75283232e-04, ...,
          5.75746162e-05, -1.76370420e-04, -3.69369634e-04],
        ...,
        [ 8.22272385e-04,  2.55492312e-04, -1.77251459e-05, ...,
          2.17828993e-03, -1.71253399e-03, -8.03825154e-04],
        [ 7.28114042e-04,  3.21790925e-04, -1.12756104e-04, ...,
          2.33058515e-03, -1.99354673e-03, -9.96105606e-04],
        [ 6.29782618e-04,  3.69876856e-04, -2.18209170e-04, ...,
          2.45313067e-03, -2.26069218e-03, -1.17605389e-03]],

       [[ 3.77360877e-04,  1.32223759e-05,  4.77482463e-05, ...,
         -6.39169375e-05,  3.30271178e-05, -3.16581660e-04],
        [ 4.80735122e-04, -2.89942283e-04,  

In [24]:
# 모델의 최종 출력 텐서 shape?? 256,14,5000 어떻게 확인??

In [25]:
src_sample.shape

TensorShape([1024, 14])

In [26]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  1574912   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  2099200   
_________________________________________________________________
dense (Dense)                multiple                  6156513   
Total params: 12,902,881
Trainable params: 12,902,881
Non-trainable params: 0
_________________________________________________________________


In [27]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val,dec_val))

Epoch 1/10
4153/4153 [==============================] - 127s 31ms/step - loss: 3.2161 - val_loss: 2.9886
Epoch 2/10
4153/4153 [==============================] - 126s 30ms/step - loss: 2.8616 - val_loss: 2.8380
Epoch 3/10
4153/4153 [==============================] - 122s 29ms/step - loss: 2.6786 - val_loss: 2.7571
Epoch 4/10
4153/4153 [==============================] - 126s 30ms/step - loss: 2.5293 - val_loss: 2.7179
Epoch 5/10
4153/4153 [==============================] - 128s 31ms/step - loss: 2.4018 - val_loss: 2.6962
Epoch 6/10
4153/4153 [==============================] - 129s 31ms/step - loss: 2.2913 - val_loss: 2.6842
Epoch 7/10
4153/4153 [==============================] - 129s 31ms/step - loss: 2.1923 - val_loss: 2.6790
Epoch 8/10
4153/4153 [==============================] - 130s 31ms/step - loss: 2.1025 - val_loss: 2.6870
Epoch 9/10
4153/4153 [==============================] - 132s 32ms/step - loss: 2.0214 - val_loss: 2.6867
Epoch 10/10
4153/4153 [==============================] 

In [28]:
# val_loss 0.2 

In [29]:
# 이부분은 잘 몰라서 노드의 코드 가져옴 

In [30]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence]) 
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [35]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> long time", max_len=20)

'<start> long time i need you to be <end> '

In [46]:
generate_text(model, tokenizer, init_sentence="<start> army can", max_len=20)

'<start> army can be your friend <end> '

In [38]:
# unk??
# loss 2.2는 되는데 val_loss는 왜 2.2가 안될까
# 문장이 끊기지 않고 이어져야하는 부분에서 끊겨버림
# 모델3개를 순서대로 돌려서 파라미터가 약 1300만개나 됨, loss가 불만족스운 경우 재설정한 모델의 실행에 시간이 많이 걸림